# Class 21:  joint entropy and the REVEAL algorithm

We'll use the bladder cancer gene expression data to test out the REVEAL algorithm. First, we'll load the data and filter to include only genes for which the median log2 expression level is > 12.  That should give us 164 genes to work with.

In [3]:
gene_matrix_for_network <- read.table("shared/bladder_cancer_genes_tcga.txt",
                                     sep="\t",
                                     header=TRUE,
                                     row.names=1,
                                     stringsAsFactors=FALSE)
print(sprintf("Dimensions of the original matrix of RNA-seq data: %d", dim(gene_matrix_for_network)))
M <- ncol(gene_matrix_for_network)
gene_matrix_filt <- gene_matrix_for_network[apply(gene_matrix_for_network, 1, median) > 12, ]

N <- nrow(gene_matrix_filt)

print(sprintf("Dimensions of the filtered gene matrix: %d", dim(gene_matrix_filt)))

[1] "Dimensions of the original matrix of RNA-seq data: 4473"
[2] "Dimensions of the original matrix of RNA-seq data: 414" 
[1] "Dimensions of the filtered gene matrix: 164"
[2] "Dimensions of the filtered gene matrix: 414"


Binarize the gene expression matrix using the mean value as a breakpoint, turning it into a NxM matrix of logicals (true/false).

In [4]:
mean_values <- apply(gene_matrix_filt, 1, mean)
gene_matrix_binarized <- (gene_matrix_filt > matrix(rep(mean_values, ncol(gene_matrix_filt)), ncol=ncol(gene_matrix_filt)))
M <- ncol(gene_matrix_filt)

The core part of the REVEAL algorithm is a function that can compute the joint entropy of a collection of binary (TRUE/FALSE) vectors X1, X2, ..., Xn (where length(X1) = length(Xi) = M).
Write a function `entropy_multiple_vecs` that takes as its input a nxM matrix (where n is the number of variables, i.e., genes, and M is the number of samples in which gene expression was measured). The function should use the log2 definition of the Shannon entropy. It should return the joint entropy H(X1, X2, ..., Xn) as a scalar numeric value. I have created a skeleton version of this function for you, in which you can fill in the code. I have also created some test code that you can use to test your function, below.

In [5]:
entropy_multiple_vecs <- function(binary_vecs) {
    stopifnot(ncol(binary_vecs) == M)  # sanity check input
    stopifnot(is.matrix(binary_vecs))  # sanity check input
    n <- nrow(binary_vecs)
    
    ## convert the nxM matrix of logicals to a Mxn data frame of factors
    df <- data.frame(t(binary_vecs))
    
    ## use "xtabs" to tabulate the number of occurrences of each combination of logical
    ## values for the n inputs; divide by M and call the resulting matrix "probmat"
    probmat <- xtabs(~., data.frame(lapply(df, factor)))/M

    ## define a matrix "hmat" that is the negative of probmat element-wise-times log2 of probmat
    hmat <- -probmat*log2(probmat)
    
    ## some entries of hmat will be NaN (do you know why?).  Set them to zero
    hmat[is.nan(hmat)] <- 0
    
    ## return the sum of entries of hmat
    sum(hmat)
}

Here are some test cases to run, after you have written your function

In [6]:
print(sprintf("Test 1 for your entropy function.  H(X,Y,Z) for three independent RVs X,Y,Z should be very close to 3: %f",
              entropy_multiple_vecs(t(replicate(3,sample(c(FALSE,TRUE), size=M, replace=TRUE))))))

print(sprintf("Test 2 for your entropy function.  H(X,Y,Z) for X,Y,Z always = FALSE, should be identically zero: %f",
              entropy_multiple_vecs(matrix(rep(FALSE, 3*M), nrow=3))))

print(sprintf("Test 3 fo ryour entropy function.  H(X1,X2,X3,X4) for the first four rows of your matrix should be 3.937690: %f",
              entropy_multiple_vecs(gene_matrix_binarized[1:4,])))

[1] "Test 1 for your entropy function.  H(X,Y,Z) for three independent RVs X,Y,Z should be very close to 3: 2.993245"
[1] "Test 2 for your entropy function.  H(X,Y,Z) for X,Y,Z always = FALSE, should be identically zero: 0.000000"
[1] "Test 3 fo ryour entropy function.  H(X1,X2,X3,X4) for the first four rows of your matrix should be 3.937690: 3.937690"


Here is an example implementation of the REVEAL algorithm using your function. It will find regulators and store regulators in the list `regulators` (the `regulators` list is initialized to NA for each gene which means that no regulators are known for that gene). Study the code and see if you can figure out exactly how it works. Note the use of `do.call`, `expand.grid`, and `apply` in order to vectorize the code, and the use of `list` and `array` in order to define data structures used in the algorithm.

In [ ]:
while(length(genes_to_fit) > 0) {
    gene_to_fit <- genes_to_fit[1]
    if (last_gene_to_fit > gene_to_fit) {
        stage <- stage + 1
        if (stage > max_stage) {
            break
        }
        print(sprintf("Getting joint entropies for combinations of %d genes", stage+1))
        entropies_for_stages[[stage+1]] <- array(apply(  expand.grid(rep(list(1:N), (stage+1))), 1, function(gene_set) {
            entropy_multiple_vecs(gene_matrix_binarized[gene_set,])
            }), dim=rep(N, stage+1))
    }
    print(sprintf("trying regulators for gene %d at stage %d  last_gene_to_fit: %f",
                  gene_to_fit, stage, last_gene_to_fit))
    possible_regulators <- setdiff(1:N, gene_to_fit)
    if (length(possible_regulators) < stage+1) {
        break
    }
    possible_regulator_combs <- matrix(combn(possible_regulators, stage), nrow=stage)
    comb_ratios <- apply(possible_regulator_combs, 2, function(reg_comb) {
        HG <- entropies_for_stages[[1]][gene_to_fit]
        entropy_G_plus_regulators <- do.call("[",c(list(entropies_for_stages[[stage+1]]),c(gene_to_fit, reg_comb)))
        entropy_of_regulators <- do.call("[",c(list(entropies_for_stages[[stage]]),reg_comb))
        ratio_res <- (HG - entropy_G_plus_regulators + entropy_of_regulators)/HG
        ratio_res
    })
    good_pred_inds <- which(comb_ratios >= ratio_thresh)
    if (length(good_pred_inds) > 0) {
        regulators_for_gene <- possible_regulator_combs[,which.max(comb_ratios)]
        print(sprintf("Regulators for gene %d are: ", gene_to_fit))
        print(regulators_for_gene)
        regulators[[gene_to_fit]] <- regulators_for_gene
        genes_to_fit <- setdiff(genes_to_fit, gene_to_fit)
    }  else {
        genes_to_fit <- c(setdiff(genes_to_fit, gene_to_fit), gene_to_fit)
    }
    last_gene_to_fit <- gene_to_fit
}

